# Conexiones e importaciones

In [57]:
import os #trabajar con el sistema operativo y variables de entorno
from dotenv import load_dotenv #cargar variable de entorno
load_dotenv() #carga las variables del entorno .env; devuelve un true o false
import spotipy #librería para trabajar con la API de Spotify
from spotipy.oauth2 import SpotifyClientCredentials #autenticación con Spotify

import pandas as pd #dataframes
import requests #peticiones a APIs
import time #librería para hacer pausas dentro de las peticiones y no saturar APIs
from urllib.parse import quote #esta librería sirve para codificar las URLs (espacios y caracteres especiales -> %20, etc)

In [46]:
#Spotify

mis_credenciales = SpotifyClientCredentials(
    client_id=os.getenv("SPOTIFY_CLIENT_ID"),
    client_secret=os.getenv("SPOTIFY_CLIENT_SECRET")
)
spotify = spotipy.Spotify(auth_manager=mis_credenciales)

In [48]:
#last fm
api_key_lastfm = os.getenv("API_KEY_LASTFM")
shared_secret_lastfm = os.getenv("SHARED_SECRET_LASTFM")

# Spotify

In [28]:
generos = sp.search(q="genre:", type='track', limit=50, offset=0)

In [53]:
generos = ["flamenco", "latin", "jazz", "rock"]
año = 2010
todas_las_canciones = []
albumes_ya_vistos = set()  # Evita duplicados: un álbum puede aparecer al buscar varios artistas (ej. colaboraciones).
datos_artistas = {}
datos_artistas = {} 
artistas_procesados = set()

print("Buscando canciones del año", año)

for genero in generos:
    print("Género:", genero)

    # 👉 ANOTACIÓN: "genre:" SOLO funciona al buscar artistas (type="artist")
    #    No funciona con álbumes ni canciones directamente.
    resultado_artistas = spotify.search(q="genre:" + genero, type="artist", limit=50)
    artistas = resultado_artistas["artists"]["items"]

    for artista in artistas:
        nombre_artista = artista["name"]

        # 👉 ANOTACIÓN: "artist:" y "year:" SÍ se pueden usar juntos
        #    al buscar álbumes (type="album") o canciones (type="track").
        #    Aquí buscamos álbumes del artista en el año indicado.
        busqueda = "artist:" + nombre_artista + " year:" + str(año)
        resultado_albumes = spotify.search(q=busqueda, type="album", limit=50)
        albumes = resultado_albumes["albums"]["items"]
        



        for album in albumes:
            id_album = album["id"]
            nombre_album = album["name"]
            
            # Si ya vimos este álbum, saltamos al siguiente
            if id_album in albumes_ya_vistos:
                continue
            
            # Si no lo habíamos visto, lo marcamos como visto
            albumes_ya_vistos.add(id_album)

            # Pedimos las canciones del álbum (esto no usa anotaciones, es una llamada directa)
            resultado_canciones = spotify.album_tracks(id_album)
            canciones = resultado_canciones["items"]

            for cancion in canciones:
                info = {
                    "nombre": cancion["name"],
                    "artista": nombre_artista,
                    "album": nombre_album,
                    "genero": genero,
                    "año": año
                }
                todas_las_canciones.append(info)

    print("  → Canciones de este género añadidas")
    print()

# Mostramos un resumen de lo encontrado
print("RESUMEN:")

# Contamos las canciones de cada género una por una
for genero in generos:
    contador = 0
    for cancion in todas_las_canciones:
        if cancion["genero"] == genero:
            contador = contador + 1
    print(f"- {genero}: {contador} canciones")

print("Total de canciones encontradas:", len(todas_las_canciones))

Buscando canciones del año 2010
Género: flamenco
  → Canciones de este género añadidas

Género: latin
  → Canciones de este género añadidas

Género: jazz
  → Canciones de este género añadidas

Género: rock
  → Canciones de este género añadidas

RESUMEN:
- flamenco: 0 canciones
- latin: 1606 canciones
- jazz: 3249 canciones
- rock: 4364 canciones
Total de canciones encontradas: 9219


In [ ]:
todas_las_canciones_df = pd.DataFrame(todas_las_canciones)
todas_las_canciones_df

In [41]:
todas_las_canciones_df.to_csv("caciones_2010.csv", index=False)

# Last FM

NameError: name 'artista_codificado' is not defined

In [54]:
if not api_key_lastfm:
    print("ERROR: La variable de entorno 'API_KEY_LASTFM' no está configurada.")
else:
    print("API Key de Last.fm cargada con éxito.")

API Key de Last.fm cargada con éxito.


In [ ]:
artista_info = []
def busqueda_info_artista(nombre_artista, api_key_lastfm):
    artista_codificado = quote(nombre_artista) #esto sirve para que los espacios y caracteres especiales no den error en la url de la api
    url_last_fm = (f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artista_codificado}&api_key={api_key_lastfm}&format=json")
    try:
        response = requests.get(url_lastfm, timeout=10)
        response.raise_for_status()
        data = response.json()
        if 'artist' in data:
            artista_info = data['artist']

NameError: name 'artistas_procesados' is not defined